In [ ]:
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
from transformers import pipeline
from tqdm import tqdm
#from tqdm.auto import tqdm
import torch
from torch.utils.data import DataLoader


import umap.umap_ as umap
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


from scipy.special import softmax
from scipy.stats import entropy

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import gc
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AutoConfig
from transformers import AutoModel
device = "cuda:0" if torch.cuda.is_available() else "cpu"

import pickle
import time

from transformers import logging
logging.set_verbosity_error()

from datasets.utils.logging import disable_progress_bar
disable_progress_bar()

from dependencies.basic_functions import *

In [ ]:
import importlib
import dependencies.basic_functions
importlib.reload(dependencies.basic_functions)

from dependencies.basic_functions import *

In [ ]:
#MERGED ALLE RESULT FILE IN /RESULTS

import os
import pickle

# Define the folder path
results_folder = './results'

# Initialize the lists
full_df_list = []
time_df_list = []
ROC_DICT_list = []

# Iterate over files in the folder
for file_name in os.listdir(results_folder):
    if file_name.startswith("results_") and file_name.endswith(".pkl"):
        file_path = os.path.join(results_folder, file_name)
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
            # Append to the respective lists
            if isinstance(data, dict) and {'full_df', 'time_df', 'ROC_DICT'}.issubset(data.keys()):
                dataset_name_tmp = data['full_df']["Dataset"].iloc[0] + ' Interpolated' #Zum Mergen mit TimeDF später
                data['time_df']["Dataset"] = dataset_name_tmp
                full_df_list.append(data['full_df'])
                time_df_list.append(data['time_df'])
                ROC_DICT_list.append(data['ROC_DICT'])
            else:
                print(f"File {file_name} does not have the expected structure.")

df = pd.concat(full_df_list, ignore_index=True)
time_result = pd.concat(time_df_list, ignore_index=True)

big_ROC_dict = {}
for roc_dict in ROC_DICT_list:
    for dataset_name, content in roc_dict.items():
        # Add the dataset_name and content to the big dictionary
        big_ROC_dict[dataset_name] = content



from scipy.interpolate import interp1d
from scipy.integrate import simps
#Interpolate -> Dann 0.0,0.05,0.1 ... 1.0 Coverage Umwandeln für Merge
#Interpolate hat probleme bei duplicates



interpolated_dataframes = []
#Method_Classifier 	Dataset 	Method 	Data_size 	Classifier
score_df = pd.DataFrame(columns=["Method_Classifier", "Dataset","Data_size","Score_50","Score_90","Score_95","Method","Classifier"])

for size in df.Data_size.unique():

    for methode_clf in df.Method_Classifier.unique():
        dff = df[df['Data_size'] == size]
        #Maybe noch 1->0.5 cov only
        dff = dff[dff['Method_Classifier'] == methode_clf]
        if dff.empty: #(Leer -> z.B. Nicht volle Runs/alle Methoden gemacht mit 10k size)
                continue
                
        methode = dff["Method"].iloc[0]
        clf_name = dff["Classifier"].iloc[0]
        
        for dataset in df.Dataset.unique():
            dffd = dff[dff['Dataset'] == dataset] #error da dataset sentiment ausgewählt wird? warum nur agnews error

            if dffd.empty: #(Leer -> z.B. Nicht volle Runs/alle Methoden gemacht mit 10k size)
                continue 

            #DROPOUT METHOD, ALLGEMEIN METHODEN DIE NICHT BIS ZUM ENDE CONVERAGE LINIE ZIEHEN AKA SICH NICHT SINNVOLL APPROXIMIEREN LASSEN
            #      ->WORKAROUND, LETZTEN ACCURACY WERT z.B. BEI 0.7 COVERAGE DANN AUCH BEI 0.0
            if methode_clf == "Dropout Agreement MV generic" or methode_clf == "Dropout Agreement DP generic":
                #print(dffd["Coverage"].values[-1], dffd["Accuracy"].values[-1])
                new_row = pd.DataFrame([{
                    "Coverage": 0.0,
                    "Accuracy": dffd["Accuracy"].values[-1]
                }])
                dffd = pd.concat([dffd, new_row], ignore_index=True)
                #print("C", dffd["Coverage"].values[-1], dffd["Accuracy"].values[-1])


            #duplicates x Achse loswerden und y dabei korrekt lassen
            df_dup = pd.DataFrame({'Coverage': dffd["Coverage"].values, 'Accuracy': dffd["Accuracy"].values})
            df_unique = df_dup.drop_duplicates(subset='Coverage', keep='first')
                            
            x,y = df_unique['Coverage'], df_unique['Accuracy']
            interp_func = interp1d(x,y, kind='linear', fill_value='extrapolate')
            #new_coverage = np.linspace(0, 1, 21)
            
            part1 = np.linspace(0, 0.9, int(0.9 / 0.05) + 1)
            part2 = np.linspace(0.91, 1, int((1 - 0.9) / 0.01) + 1)
            new_coverage = np.concatenate((part1, part2)) #Coverage in 1% bis 0.9 dann in 5%
            
            interpolated_accuracy = interp_func(new_coverage)
            
            #####SCORE BERECHNUNG -> AREA UNDER CURVE APPROX mit Simps (zeiteffizient, non kontinuirliche Funktion...)

            new_coverage_for_integration_x = np.linspace(0.5, 1.0, 1000)
            Score_50 = simps(interp_func(new_coverage_for_integration_x), new_coverage_for_integration_x) 
            new_coverage_for_integration_x = np.linspace(0.9, 1.0, 1000)
            Score_90 = simps(interp_func(new_coverage_for_integration_x), new_coverage_for_integration_x)
            new_coverage_for_integration_x = np.linspace(0.95, 1.0, 1000)
            Score_95 = simps(interp_func(new_coverage_for_integration_x), new_coverage_for_integration_x)
            
            ####
            
            if np.isnan(interpolated_accuracy).any():
                print(methode_clf,dataset,"ERROR")

            data = {
                'Accuracy': interpolated_accuracy,
                'Coverage': new_coverage,
                #'Score' : [area]*len(new_coverage),
                "Method_Classifier": [methode_clf]*len(new_coverage)
            }
            
            dataset_interpol = dataset + " Interpolated"
            new_interpolate = create_pandas_frame(data,dataset_interpol,methode,size,clf_name,alldata="True")
            interpolated_dataframes.append(new_interpolate)
            #"Method_Classifier", "Dataset", "Score","Method","Classifier"
            score_df.loc[len(score_df)] = [methode_clf,dataset_interpol,size,Score_50,Score_90,Score_95,methode,clf_name]
        
        

interpolate_df = pd.concat(interpolated_dataframes, ignore_index=True)
full_result_df = pd.concat([interpolate_df,df], ignore_index=True) #Berechneter Stuff + Raw Datensätze

#Scores - Baseline (Most Frequent Class) + Stanartisieren
#score_df
score_df["Score_50_norm"] = score_df["Score_50"] #copy zum ersetzen
score_df["Score_90_norm"] = score_df["Score_90"]
score_df["Score_95_norm"] = score_df["Score_95"]

for size in score_df.Data_size.unique():
    for dataset in score_df.Dataset.unique(): # NUR FÜR INTERPOLATED ANWENDEN!
        for scoretype in ["Score_50","Score_90","Score_95"]:
            baseline_score = score_df[(score_df.Data_size == size) & (score_df.Dataset == dataset) & (score_df.Method == "Most Frequent Class")][scoretype]
            if baseline_score.empty:
                continue
            baseline_score = baseline_score.values[0]
            condition = (score_df.Data_size == size) & (score_df.Dataset == dataset)
            score_df.loc[condition, scoretype] -= baseline_score
            
            #standartisieren
            scores = score_df[(score_df.Data_size == size) & (score_df.Dataset == dataset)][scoretype].values
            max_deviation = np.max(np.abs(scores))
            standardized_scores = scores / max_deviation if max_deviation != 0 else scores
            score_df.loc[(score_df.Data_size == size) & (score_df.Dataset == dataset), f"{scoretype}_norm"] = standardized_scores
            #score_df.loc[(score_df.Data_size == size) & (score_df.Dataset == dataset), scoretype] = standardized_scores


#Ranking Tabelle

#noch für alle DATASET und Datasize
pd_rank_list = []
for dataset in score_df.Dataset.unique():
    pd_ranking = pd.DataFrame(columns=["Dataset","Method_50","Method_90","Method_95","Score_50_norm","Score_90_norm","Score_95_norm"])

    for num in ["50","90","95"]:
        scoretype = f"Score_{num}_norm"
        methodtype = f"Method_{num}"

        part = score_df[(score_df.Dataset ==dataset)&(score_df.Data_size ==max(full_result_df.Data_size.unique()))]
        part = part.reset_index(drop=True)
        sorted_index = np.argsort(part[scoretype])

        sorted_score= part[scoretype][sorted_index].values
        sorted_method = part["Method_Classifier"][sorted_index].values
        pd_ranking[scoretype] = sorted_score
        pd_ranking[methodtype] = sorted_method
        pd_ranking["Dataset"] = dataset
        #print(sorted_index)

    pd_ranking = pd_ranking.iloc[::-1].reset_index(drop=True)
    pd_rank_list.append(pd_ranking)
    
pd_ranking_full = pd.concat(pd_rank_list, ignore_index=True)


In [ ]:
with open('evaluation_ranking_full.pkl', 'wb') as f:
    pickle.dump(pd_ranking_full, f)

with open('evaluation_full.pkl', 'wb') as f:
    pickle.dump(full_result_df, f)

time_result.rename(columns={'Method': 'Method_Classifier'}, inplace=True)
score_time_df = pd.merge(score_df, time_result, on=['Method_Classifier', 'Data_size', 'Dataset'], how='inner')
with open('evaluation_time_full.pkl', 'wb') as f:
    pickle.dump(score_time_df, f)

with open('evaluation_roc.pkl', 'wb') as f:
    pickle.dump(big_ROC_dict, f)